# Dataset : Penn-Fudan Database for Pedestrian Detection and Segmentation

* Url : https://www.cis.upenn.edu/~jshi/ped_html/

* 이미지에서 gradient 개념을 이용하여 feature vector를 정의하고, SVM 알고리즘을 이용하여 보행자 위치를 검출하는 알고리즘 HOG 알고리즘 공부

* image에서 gradient란?
  * 이미지는 수많은 픽셀로 이루어져있고 픽셀들이 서로 붙어있는데, 그 붙어있는 pixel끼리 각각의 밝기 값(gray scale 값)이 똑같을 수도 있고 다를 수도 있다.
  * 그러면 이때 그 pixel 좌표 값 차이 대비 밝기 값이 얼만큼 변했는가가 image에서 gradient라 할 수 있을 것
  * 이러한 원리로 이미지에서 edge 부분이 물체의 중심 부분보다 gradient가 크다


# Histogram of Oriented Gradients(HOG)
  * Histogram of Oriented Gradient(HOG) 에서는 영상의 feature를 기본적으로
방향에 따른 gradient 값의 histogram 으로 표현
  * 보행자 검출(Pedestrian Detection) 목적을 위한 HOG는 기본적으로 64x128 크기의 영상 사용
  * 동작원리
    1. 입력 영상으로부터 각 pixel의 Gradient의 크기와 방향을 계산
    2. 8x8 pixel로 묶은 단위를 cell이라고 한다 ->  64x128 영상에서 cell은 가로 방향으로 8(64/8)개, 세로 방향으로 16(128/8)개 생성
    3. 각각의 cell로부터 Gradient의 방향에 대한 Gradient 크기 히스토그램을 구한다
    4. 인접한 4개의 cell을 합쳐서 block이라고 정의
       
       즉, 8x8pixel 은 cell → cell을 4개로 묶으면 하나의 block (16x16pixel) 이 된다
    5. block 안 cell의 histogram을 연결시킨 후 L1,L2 정규화를 하여 최종적인 영상의 feature vector를 완성시킨다.
    6. 이미지(feature vector)과 label로 이루어진 쌍(example)은 선형 SVM 분류기를 통해 학습된다.
    7. test 이미지를 주면 feature vector를 가지고 detection window에서 내가 찾고자 하는 label이 있는지를 확인하고, 그 window의 위치를 알려주는 것이다.

In [2]:
from __future__ import print_function
from imutils.object_detection import non_max_suppression
from imutils import paths
import numpy as np
import argparse
import imutils
import cv2

import matplotlib.pyplot as plt

In [3]:
images_path = '/content/drive/MyDrive/study 공부기록/PennFudanPed/PNGImages'

hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

In [9]:
for imagePath in paths.list_images(images_path):
    image = cv2.imread(imagePath)
    image = imutils.resize(image, width=min(400, image.shape[1]))
    orig = image.copy()
    # detect people in the image(hog와 svm 동시에 적용하여 feature extraction과 classification 수행)
    (rects, weights) = hog.detectMultiScale(image, winStride = (4,4), padding=(8,8), scale=1.05)
    # draw bounding box
    for (x,y,w,h) in rects:
        cv2.rectangle(orig, (x,y), (x+w, y+h), (0,0,255), 2)
    rects = np.array([[x,y,x+w,y+h] for (x,y,w,h) in rects])
    pick = non_max_suppression(rects, probs=None, overlapThresh=0.65)
    for (xA,yA,xB,yB) in pick:
        cv2.rectangle(image, (xA, yA), (xB, yB), (0,255,0),2)
    filename = imagePath[imagePath.rfind('/') + 1:]
    print('[INFO] {}: {} original boxes, {} after suppression'.format(filename, len(rects), len(pick)))
    plt.subplot(1,2,1)
    plt.imshow(orig[:,:,::-1])
    plt.title('Before NMS')
    plt.subplot(1,2,2)
    plt.imshow(image[:,:,::-1])
    plt.title('After NMS')
    plt.show()

Output hidden; open in https://colab.research.google.com to view.